In [8]:
import pandas as pd 
import numpy as np 
import time 

In [9]:
import collections
import random
import torch
from torch import nn
from torch.nn import functional as F

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [136]:
class SeqDataLoader:  
    """序列数据迭代器"""
    def __init__(self, corpus, batch_size, num_steps, vocab_size):
        self.corpus = corpus
        self.batch_size, self.num_steps = batch_size, num_steps
        self.vocab_size = vocab_size

    def __iter__(self):
        return self.seq_data_iter_sequential(self.corpus, self.batch_size, self.num_steps, self.vocab_size)
    
    def seq_data_iter_sequential(self, corpus, batch_size, num_steps, vocab_size):
        """顺序生成一个小批量子序列"""
        # 从随机偏移量开始划分序列
        offset = random.randint(0, num_steps)
        num_tokens = ((len(corpus) - offset - 1) // batch_size) * batch_size
        Xs = torch.tensor(corpus[offset: offset + num_tokens], dtype=torch.float32)
        Ys = torch.tensor(corpus[offset + 1: offset + 1 + num_tokens], dtype=torch.float32)
        Xs, Ys = Xs.reshape(batch_size, -1, vocab_size), Ys.reshape(batch_size, -1, vocab_size)
        num_batches = Xs.shape[1] // num_steps
        for i in range(0, num_steps * num_batches, num_steps):
            X = Xs[:, i: i + num_steps]
            Y = Ys[:, i: i + num_steps][:, :, 0]
            yield X, Y

In [137]:
class RNNModel(nn.Module):
    """循环神经网络模型"""
    def __init__(self, rnn_layer, vocab_size, num_item, num_place, item_embed_size=5, place_embed_size=5, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        if not self.rnn.bidirectional:
            self.num_directions = 1
            self.linear = nn.Linear(self.num_hiddens, self.vocab_size)
        else:
            self.num_directions = 2
            self.linear = nn.Linear(self.num_hiddens * 2, self.vocab_size)
        # item 和 place的embedding
        self.num_item = num_item
        self.num_place = num_place
        self.item_embed_size = item_embed_size
        self.place_embed_size = place_embed_size
        self.item_embed = nn.Embedding(num_embeddings=num_item, embedding_dim=item_embed_size)
        self.place_embed = nn.Embedding(num_embeddings=num_place, embedding_dim=place_embed_size)
        

    def forward(self, inputs, iid, pid, state):
        iid_tensor = torch.full((inputs.shape[0], inputs.shape[1], 1), iid)
        pid_tensor = torch.full((inputs.shape[0], inputs.shape[1], 1), pid)
        iid_embed = self.item_embed(iid_tensor).view(inputs.shape[0], inputs.shape[1], self.item_embed_size)
        pid_embed = self.place_embed(pid_tensor).view(inputs.shape[0], inputs.shape[1], self.place_embed_size)
        X = torch.cat((inputs, iid_embed, pid_embed), dim=-1)
        # 时间步数*批量大小,隐藏单元数
        X.transpose_(0, 1)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        # 将Y的形状改为(时间步数*批量大小,隐藏单元数)
        # 输出(时间步数*批量大小,词表大小)。
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state

    def begin_state(self, device, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
            # nn.GRU以张量作为隐状态
            return  torch.zeros((self.num_directions * self.rnn.num_layers,
                                 batch_size, self.num_hiddens),
                                device=device)
        else:
            # nn.LSTM以元组作为隐状态
            return (torch.zeros((
                self.num_directions * self.rnn.num_layers,
                batch_size, self.num_hiddens), device=device),
                    torch.zeros((
                        self.num_directions * self.rnn.num_layers,
                        batch_size, self.num_hiddens), device=device))

In [138]:
def grad_clipping(net, theta):  #@save
    """裁剪梯度"""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

def train_epoch(net, train_iter, iid, pid, loss, updater, device, use_random_iter):
    """训练网络一个迭代周期"""
    state = None
    start_time = time.time()
    mse = 0
    size = 0
    for X, Y in train_iter:
        if state is None or use_random_iter:
            # 在第一次迭代或使用随机抽样时初始化state
            state = net.begin_state(batch_size=X.shape[0], device=device)
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # state对于nn.GRU是个张量
                state.detach_()
            else:
                for s in state:
                    s.detach_()
                    
        # y = Y.reshape(-1)
        y = Y.transpose(0, 1).reshape(-1)
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, iid, pid, state)
        l = loss(y_hat.reshape(-1), y)
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            updater(batch_size=1)
        mse += l * y.numel()
        size += y.numel()
    
    return mse / size, time.time() - start_time

def train(net, train_iter, iid, pid, lr, num_epochs, device, use_random_iter=False):
    """训练模型"""
    loss = nn.MSELoss()
    # 初始化
    updater = torch.optim.Adam(net.parameters(), lr)
    # 训练
    for epoch in range(num_epochs):
        epochmse, speed = train_epoch(
            net, train_iter, iid, pid, loss, updater, device, use_random_iter)
        if epoch == 0:
            print(f'epoch: {epoch + 1}, mse: {epochmse}, time: {speed}')
        if (epoch + 1) % 100 == 0:
            print(f'epoch: {epoch + 1}, mse: {epochmse}, time: {speed}')

In [139]:
def predict_rnn(inputs, iid, pid, num_preds, net, device):
    """向后预测"""
    state = net.begin_state(batch_size=1, device=device)
    outputs = inputs.flatten().tolist() 
    # 预热
    Y, state = net(inputs, iid=iid, pid=pid, state=state)
    # 向后第一步
    outputs.append(Y.flatten()[-1].item())
    for _ in range(num_preds - 1):  # 预测num_preds步
        Y, state = net(torch.tensor(outputs[-1]).view(1, 1, 1), iid=iid, pid=pid, state=state)
        outputs.append(Y.item())
    return outputs

# Test

In [162]:
tsdata = pd.DataFrame({'sale': np.array([1,3,5,7,9,7,5,3] * 1000)})
# for i in range(1, 9):
#     tsdata['pre_{}'.format(i)] = tsdata['sale'].shift(i)
# tsdata.dropna(inplace=True)
iid = 0
pid = 0
data = tsdata.to_numpy()
scaler = MinMaxScaler() 
data = scaler.fit_transform(data)

In [163]:
batch_size, num_steps = 128, 10
num_inputs = 1 + 3 + 3
num_hiddens = 16
net = RNNModel(nn.LSTM(num_inputs, num_hiddens), vocab_size=1, num_item=2, num_place=2, item_embed_size=3, place_embed_size=3)

In [164]:
for name, param in net.named_parameters():
    print(name, param.size())

rnn.weight_ih_l0 torch.Size([64, 7])
rnn.weight_hh_l0 torch.Size([64, 16])
rnn.bias_ih_l0 torch.Size([64])
rnn.bias_hh_l0 torch.Size([64])
linear.weight torch.Size([1, 16])
linear.bias torch.Size([1])
item_embed.weight torch.Size([2, 3])
place_embed.weight torch.Size([2, 3])


In [165]:
# 训练
train_iter = SeqDataLoader(data, batch_size=batch_size, num_steps=num_steps, vocab_size=1)
num_epochs, lr = 500, 0.01
device = 'cpu'

In [166]:
train(net, train_iter, iid, pid, lr, num_epochs, device)

epoch: 1, mse: 0.20028053224086761, time: 0.04900360107421875
epoch: 100, mse: 0.0012313289334997535, time: 0.05304884910583496
epoch: 200, mse: 0.0011149842757731676, time: 0.05504727363586426
epoch: 300, mse: 0.0007263133302330971, time: 0.04502129554748535
epoch: 400, mse: 0.0006801017443649471, time: 0.05305743217468262
epoch: 500, mse: 0.0010754867689684033, time: 0.05202531814575195


In [171]:
prefix = torch.tensor(data[-15:]).reshape(1, -1, 1)
predict = predict_rnn(inputs=prefix, iid=iid, pid=pid, num_preds=15, net=net, device='cpu')

In [172]:
scaler.inverse_transform(np.array(predict).reshape(-1, 1))

array([[3.        ],
       [5.        ],
       [7.        ],
       [9.        ],
       [7.        ],
       [5.        ],
       [3.        ],
       [1.        ],
       [3.        ],
       [5.        ],
       [7.        ],
       [9.        ],
       [7.        ],
       [5.        ],
       [3.        ],
       [1.02027948],
       [3.03399801],
       [5.04173279],
       [7.09666777],
       [9.10932255],
       [7.01463938],
       [5.01331282],
       [3.04564166],
       [1.04226877],
       [3.02850676],
       [5.03665257],
       [7.08926201],
       [9.10710621],
       [7.01918173],
       [5.01760435]])

In [21]:
import pandas as pd 

def gene(start_date, end_date, gid, pid):
    dr = pd.date_range(start_date, end_date)
    n = len(dr)
    qty = np.arange(n) / 50 + np.random.randint(-3, 3, n)
    df = pd.DataFrame({'date': dr, 
                      'goodsqty': qty,
                      'goodsid': gid,
                      'placepointid': pid})
    return df

# 根据goodsid， placepointid 获取对应的时间序列数据
def get_subdata(data, gid, pid):
    return data[(data['goodsid'] == gid) & (data['placepointid'] == pid)]
    
# 生成虚拟数据, 四个组合 (0, 0),(0, 1),(1, 0),(1, 1)
data = pd.concat((gene('1997-1-1', '1999-4-5', 0, 0), 
           gene('2020-1-1', '2022-3-3', 0, 1),
           gene('2015-12-12', '2018-9-9', 1, 0),
           gene('2025-6-9', '2027-9-9', 1, 1)), axis=0)

# 选出 goodsid = 0, placepointid = 0 的数据
subdata = get_subdata(data, 1, 0)

# 对日期排序
subdata = subdata.sort_values(by=['date'])

# 对subdata fit 一个ts模型 ......
subdata

,date,goodsqty,goodsid,placepointid
0,2015-12-12,0.00,1,0
1,2015-12-13,-0.98,1,0
2,2015-12-14,-2.96,1,0
3,2015-12-15,-0.94,1,0
4,2015-12-16,2.08,1,0
...,...,...,...,...
998,2018-09-05,19.96,1,0
999,2018-09-06,19.98,1,0
1000,2018-09-07,18.00,1,0
1001,2018-09-08,20.02,1,0
